# Model 1: MultiNomial Naive Bayes

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## Loading the Dataset

In [ ]:
data = pd.read_csv('/Users/tahafaisal/Desktop/ml-news-classification/data/FINAL_DATASET.csv')

data.info()
print()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2401 entries, 0 to 2400
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2401 non-null   int64 
 1   title       2401 non-null   object
 2   link        2401 non-null   object
 3   content     2401 non-null   object
 4   gold_label  2401 non-null   object
dtypes: int64(1), object(4)
memory usage: 93.9+ KB



,id,title,link,content,gold_label
0,0,ادارہ 'علم دوست' کی جانب سے'معلوماتِ عامہ' کے ...,https://www.express.pk/story/2733338/idara-ilm...,ادارہ 'علم دوست' کی جانب سے'معلوماتِ عامہ' کے ...,Entertainment
1,1,فلم ساز ودھو ونود چوپڑا کی نئی فلم ’زیرو سے ری...,https://www.express.pk/story/2733336/director-...,معروف فلم ساز ودھو ونود چوپڑا نے اپنی نئی فلم ...,Entertainment
2,2,عمر بڑھنے کے ساتھ وزن کم کرنا مشکل ہوتا ہے، اب...,https://www.express.pk/story/2733331/umer-barh...,ابھیشیک بچن نے اپنی نئی فلم ’آئی وانٹ ٹو ٹاک‘ ...,Entertainment
3,3,ملائکہ اروڑا والد کے انتقال کے بعد کام پر واپس...,https://www.express.pk/story/2733327/malaikaar...,مشہور اداکارہ ملائکہ اروڑا حال ہی میں والد کے ...,Entertainment
4,4,ڈائریکٹر دھرمیش درشن کا دو بار شاہ رخ خان کی ف...,https://www.express.pk/story/2733325/directord...,بالی ووڈ کے معروف ہدایتکار دھرمیش درشن نے حالی...,Entertainment


## Preprocessing the Dataset

In [5]:

with open('/Users/tahafaisal/Desktop/ml-news-classification/data/stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

def preprocessor(text):
    
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text

data['title'] = data['title'].apply(preprocessor)
data['content'] = data['content'].apply(preprocessor)

data['combined'] = data['title'] + " " + data['content']

data.head()


,id,title,link,content,gold_label,combined
0,0,ادارہ 'علم دوست' جانب سے'معلوماتِ عامہ' ماہرین...,https://www.express.pk/story/2733338/idara-ilm...,ادارہ 'علم دوست' جانب سے'معلوماتِ عامہ' ممتاز ...,Entertainment,ادارہ 'علم دوست' جانب سے'معلوماتِ عامہ' ماہرین...
1,1,فلم ساز ودھو ونود چوپڑا نئی فلم ’زیرو سے ری اس...,https://www.express.pk/story/2733336/director-...,معروف فلم ساز ودھو ونود چوپڑا نے اپنی نئی فلم ...,Entertainment,فلم ساز ودھو ونود چوپڑا نئی فلم ’زیرو سے ری اس...
2,2,عمر بڑھنے ساتھ وزن کم کرنا مشکل ہوتا ہے، ابھیش...,https://www.express.pk/story/2733331/umer-barh...,ابھیشیک بچن نے اپنی نئی فلم ’آئی وانٹ ٹو ٹاک‘ ...,Entertainment,عمر بڑھنے ساتھ وزن کم کرنا مشکل ہوتا ہے، ابھیش...
3,3,ملائکہ اروڑا والد انتقال بعد کام واپس، والد نا...,https://www.express.pk/story/2733327/malaikaar...,مشہور اداکارہ ملائکہ اروڑا حال میں والد انتقال...,Entertainment,ملائکہ اروڑا والد انتقال بعد کام واپس، والد نا...
4,4,ڈائریکٹر دھرمیش درشن کا بار شاہ رخ خان فلمیں ٹ...,https://www.express.pk/story/2733325/directord...,بالی ووڈ معروف ہدایتکار دھرمیش درشن نے حالیہ گ...,Entertainment,ڈائریکٹر دھرمیش درشن کا بار شاہ رخ خان فلمیں ٹ...


In [6]:
X = data['content'] 
y = data['gold_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(1920,) (1920,)
(481,) (481,)


## Implementation of Multinomial Naive Bayes From Scratch

In [7]:
class BagOfWords:
    def __init__(self):
        self.vocabulary = {}  
        self.vocab_size = 0

    def fit(self, corpus):

        unique_words = set()

        for sentence in corpus:
            words = sentence.split() 
            unique_words.update(words)  

        self.vocabulary = {word: idx for idx, word in enumerate(sorted(unique_words))}
        self.vocab_size = len(self.vocabulary)

    def vectorize(self, sentence):

        vector = [0] * self.vocab_size
        
        words = sentence.split()

        for word in words:
            if word in self.vocabulary:
                index = self.vocabulary[word]
                vector[index] += 1

        return vector

In [8]:
model = BagOfWords()
model.fit(X_train)

vector_training = [model.vectorize(doc) for doc in X_train]
vector_test = [model.vectorize(doc) for doc in X_test]


In [9]:
class MultiNaiveBayes:
    def __init__(self):
        self.classprobs = {}  
        self.wordprobs = {}  
        self.vocab_size = 0    
        self.bow = BagOfWords()  # Assume BagOfWords handles vectorization without LabelEncoder                

    def fit(self, X, y):
        self.bow.fit(X)         
        X_vectorized = [self.bow.vectorize(doc) for doc in X] 
        self.vocab_size = self.bow.vocab_size 
        n_docs = len(X)
        
        # Get unique classes as they are
        unique_classes = np.unique(y)  
        class_counts = {c: 0 for c in unique_classes}
        word_counts = {c: np.zeros(self.vocab_size) for c in unique_classes}

        for i in range(n_docs):
            c = y.iloc[i]  # Updated to y.iloc[i] to handle pandas Series correctly
            class_counts[c] += 1
            word_counts[c] += X_vectorized[i]

        for c in unique_classes:
            self.classprobs[c] = class_counts[c] / n_docs
            total_words_in_class = np.sum(word_counts[c])
            self.wordprobs[c] = (word_counts[c] + 1) / (total_words_in_class + self.vocab_size)
    
    def predict(self, X):
        X_vectorized = np.array([self.bow.vectorize(doc) for doc in X])
        predictions = []
        
        for doc_vec in X_vectorized:
            class_scores = {}
            for c in self.classprobs:
                log_prob_c = np.log(self.classprobs[c])
                log_prob_x_given_c = np.sum(doc_vec * np.log(self.wordprobs[c]) + (1 - doc_vec) * np.log(1 - self.wordprobs[c]))
                class_scores[c] = log_prob_c + log_prob_x_given_c
            
            best_class = max(class_scores, key=class_scores.get)
            predictions.append(best_class)
        
        return predictions


In [10]:
model = MultiNaiveBayes()
model.fit(X_train, y_train) 

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
confusionmatrix = confusion_matrix(y_test, predictions)

print("Manually Implemented Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)

Manually Implemented Naive Bayes:
Accuracy:  0.9771309771309772
Precision:  0.9768169034294732
Recall:  0.9776858471875268
F1 Score:  0.977047259538003
[[ 99   0   1   4   0]
 [  1  91   1   0   1]
 [  0   0  88   0   0]
 [  0   1   2  92   0]
 [  0   0   0   0 100]]


In [11]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(vector_training, y_train)
mnbprediction = model.predict(vector_test)

accuracy = accuracy_score(y_test, mnbprediction)
precision = precision_score(y_test, mnbprediction, average='macro')
recall = recall_score(y_test, mnbprediction, average='macro')
f1 = f1_score(y_test, mnbprediction, average='macro')
confusionmatrix = confusion_matrix(y_test, mnbprediction)

print("For Sklearn MultiNomial Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)
print()

For Sklearn MultiNomial Naive Bayes:
Accuracy:  0.9771309771309772
Precision:  0.9768169034294732
Recall:  0.9776858471875268
F1 Score:  0.977047259538003
[[ 99   0   1   4   0]
 [  1  91   1   0   1]
 [  0   0  88   0   0]
 [  0   1   2  92   0]
 [  0   0   0   0 100]]

